In [13]:
import os
os.getcwd()

'C:\\Users\\USER\\CNN Assignment'

In [2]:
'''########################## IMAGE PRE-PROCESSING for TRAINING and TESTING data ##############################'''
TrainingImagePath = 'D:\\Hashir\\IVY\\Python\\CNN-Pneumonia\\Train'

from keras.preprocessing.image import ImageDataGenerator

# Defining pre-processing transformations on raw images of training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 240 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


{'NORMAL': 0, 'PNEUMONIA': 1}

### Creating a list of faces and classes

In [5]:
Trainclasses=training_set.class_indices

Resultmap={}
for facevalue, facename in zip(Trainclasses.values() , Trainclasses.keys()):
    Resultmap[facevalue]=facename
    
import pickle 
with open('D:\\Hashir\\IVY\\Python\\CNN-Pneumonia\\Resultmap.pkl', 'wb') as f:
    pickle.dump(Resultmap, f, pickle.HIGHEST_PROTOCOL)
    
print('Mapping X-ray and its Type :', Resultmap)

Outputneurons=len(Resultmap)
print('\n No.of. neurons:', Outputneurons)

Mapping X-ray and its Type : {0: 'NORMAL', 1: 'PNEUMONIA'}

 No.of. neurons: 2


### Creating the CNN model

In [6]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

classifier= Sequential()

classifier.add(Convolution2D(32, kernel_size=(5,5), strides=(1,1), input_shape=(64,64,3), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
#adding another layer for accuracy
classifier.add(Convolution2D(32, kernel_size=(5,5), strides=(1,1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))

classifier.add(Flatten())
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(Outputneurons, activation='softmax'))

classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


### Fitting the model

In [7]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=5,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=5)

EndTime=time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
5/5 [==============================] - 5s 1s/step - loss: 0.7322 - accuracy: 0.5139 - val_loss: 0.6623 - val_accuracy: 0.5125
Epoch 2/10
5/5 [==============================] - 5s 949ms/step - loss: 0.6573 - accuracy: 0.6875 - val_loss: 0.6283 - val_accuracy: 0.8125
Epoch 3/10
5/5 [==============================] - 5s 1s/step - loss: 0.6333 - accuracy: 0.6562 - val_loss: 0.7017 - val_accuracy: 0.4313
Epoch 4/10
5/5 [==============================] - 5s 951ms/step - loss: 0.6023 - accuracy: 0.6458 - val_loss: 0.4915 - val_accuracy: 0.8813
Epoch 5/10
5/5 [==============================] - 5s 951ms/step - loss: 0.5544 - accuracy: 0.6667 - val_loss: 0.4352 - val_accuracy: 0.8687
Epoch 6/10
5/5 [==============================] - 5s 1s/step - loss: 0.4998 - accuracy: 0.7431 - val_loss: 0.4477 - val_accuracy: 0.7937
Epoch 7/10
5/5 [==============================] - 5s 987ms/step - loss: 0.4652 - accuracy: 0.

### Saving the model

In [8]:
## Saving the model
classifier.save('D:\\Hashir\\IVY\\Python\\CNN-Pneumonia\\classifier.pkl')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: D:\Hashir\IVY\Python\CNN-Pneumonia\classifier.pkl\assets


### Testing the model on a different face

In [10]:
# NORMAL

import numpy as np
from keras.preprocessing import image

testImage="D:\\Hashir\\IVY\\Python\\CNN-Pneumonia\\Test\\NORMAL\\IM-0021-0001.jpeg"
test_image=image.load_img(testImage,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  NORMAL


In [12]:
# PNEUMONIA 

testImage="D:\\Hashir\\IVY\\Python\\CNN-Pneumonia\\Test\\PNEUMONIA\\person11_virus_38.jpeg"
test_image=image.load_img(testImage,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  PNEUMONIA
